## Bert Tokanizer

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Preprocess

In [6]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
df = pd.read_csv("Data/malicious_phish.csv").sample(100000)
df["type"] = pd.Categorical(df["type"]).codes
X = df["url"]
y = df["type"]

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X.values,y.values,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((80000,), (20000,), (80000,), (20000,))

In [45]:
import swifter
df["url"][:1].swifter.apply(lambda text: tokenizer.encode_plus(
            text,
            add_special_tokens=True,max_length=128,
            return_token_type_ids=False,return_attention_mask=False,
            padding=True,truncation=True
        )["input_ids"])[305990]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

[101,
 2599,
 23632,
 2487,
 1012,
 4012,
 1013,
 2982,
 1013,
 2249,
 1013,
 2047,
 1011,
 2259,
 1011,
 2103,
 1012,
 16129,
 102]

In [29]:
# validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
dataset_to_numpy = list(validation_data.as_numpy_iterator())
shape = tf.shape(dataset_to_numpy)
print(shape)
# validation_data = validation_data.batch(32)

ValueError: Attempt to convert a value ({'input_ids': array([  101,  2998,  2850,  2696,  1012,  1047,  9189,  2226,  1012,
        4012,  1013,  2374,  1013,  5088,  1011,  2780,  1012,  2004,
        2361,  2595,  1029,  3931,  1027,  1013,  2951,  1013,  5088,
        1013,  2780,  1013,  2136, 23833,  1012, 16129,   102]), 'attention_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'token_type_ids': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}) with an unsupported type (<class 'dict'>) to a Tensor.

In [35]:
dataset_to_numpy[0][0]["input_ids"].shape

(35,)

In [36]:
validation_data = validation_data.batch(32)

### Model
we will not finetune the bert classifier at first

In [27]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(20),input_shape=(35,))

### Metrics

In [ ]:
test = pd.DataFrame({DATA_COLUMN:x_test,LABEL_COLUMN:y_test})
test_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
test_data = convert_examples_to_tf_dataset(list(test_InputExamples), tokenizer)

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict(test_data)
print(classification_report(y_test,pred))